In [1]:
import numpy as np
from sequence_models.pretrained import load_model_and_alphabet

In [2]:
model, collater = load_model_and_alphabet('../pretrained_esm/microsoft/carp_600k.pt')

# maybe download others and test them

In [4]:
seqs = [['MDREQ'], ['MGTRRLLP']]
# seqs = [['MGTRRLLP']]
seqs = [['MDREQ']]
x = collater(seqs)[0]  # (n, max_len)
rep = model(x)  # (n, max_len, d_model)

In [5]:
rep['representations'][16][0].shape

torch.Size([5, 128])

In [14]:
seqs = [['MDREQ']]
# seqs = [['MGTRRLLP']]
# seqs = [['MDREQ']]
x = collater(seqs)[0]  # (n, max_len)
rep = model(x)  # (n, max_len, d_model)
rep['representations'][16].shape

torch.Size([1, 5, 128])

In [1]:
def metrics(y_pred, y_true):
    true_positive = 0
    true_negative = 0
    false_negative = 0
    false_positive = 0

    for i in range(len(y_pred)):
        window = y_true[max(0, i-20):min(len(y_true), i+20)]
        if y_pred[i] == 1.0:
            if 1.0 in window:
                true_positive += 1
            else:
                false_positive += 1


        elif y_pred[i] == 0.0:
            if y_true[i] == 1.0:
                false_negative += 1
            else:
                true_negative += 1

    try:
        accuracy = (true_negative + true_positive) / (true_negative + true_positive + false_negative + false_positive)
    except ZeroDivisionError:
        accuracy = 0

    try:
        precision = true_positive / (true_positive + false_positive)
    except ZeroDivisionError:
        precision = 0

    try:
        recall = true_positive / (true_positive + false_negative)
    except ZeroDivisionError:
        recall = 0

    try:
        f1 = (2 * precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0

    try:
        mcc = ((true_positive * true_negative) - (false_positive * false_negative)) / ((true_positive + false_positive) * (true_positive + false_negative) * (true_negative + false_positive) * (true_negative + false_negative))**0.5
    except ZeroDivisionError:
        mcc = 0

    return (accuracy, precision, recall, f1, mcc)

In [5]:
pred = [0] * 10000000
true = [0] * 10000000

pred[4] = 1
true[5] = 1

metrics(pred, true)

(0.9999999, 1.0, 0.5, 0.6666666666666666, 0.707106745831204)